# Shakespearian Dialog

Let's try to use a Large Language Model to implement a project where two chatbots are speaking to each other as comic characters from Shakespeare 

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

## Use an efficient, distilled GPT2 model

In [2]:
model_name = "distilgpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [4]:
def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, num_exchange=5, max_response_length=50):
    """
    Simulates a conversation between two Shakespearean characters, with each response restricted in length.

    Args:
        character1 (str): Name of the first character.
        character2 (str): Name of the second character.
        start_sentence (str): The initial sentence to start the conversation.
        model: Pre-trained GPT-2 model.
        tokenizer: GPT-2 tokenizer.
        num_exchange (int): Number of exchanges between the characters.
        max_response_length (int): Maximum length of each response.

    Returns:
        str: The simulated conversation.
    """
    conversation = f"{character1}: {start_sentence}\n"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        input_ids = tokenizer.encode(conversation, return_tensors='pt')
        # Set the maximum length for the model generation
        max_length_for_generation = len(input_ids[0]) + max_response_length
        output = model.generate(input_ids, max_length=max_length_for_generation, pad_token_id=tokenizer.eos_token_id)
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        last_newline = response.rfind('\n')
        last_response = response[last_newline:].strip()  # Extract only the last response

        conversation_response = f"{current_character}: {last_response}"
        print(conversation_response)
        conversation += conversation_response + "\n"
        current_character = character1 if current_character == character2 else character2

    return conversation



# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", 
                                  model, tokenizer)
print(conversation)


Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: The first thing I want to say to you is that I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not
Bottom: Falstaff: I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am
Falstaff: Bottom: Falstaff: I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game.
Bottom: Falstaff: Bottom: Falstaff: I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of
Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: The first thing I want to say to you is that I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not
Bottom: Falstaff: I

In [5]:
def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, num_exchange=5, max_response_length=50):
    """
    Simulates a conversation between two Shakespearean characters, with improved generation to avoid repetition.

    Args:
        character1, character2, start_sentence, model, tokenizer: Same as before.
        num_exchange (int): Number of exchanges between the characters.
        max_response_length (int): Maximum length of each response.

    Returns:
        str: The simulated conversation.
    """
    conversation = f"{character1}: {start_sentence}\n"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        input_ids = tokenizer.encode(conversation, return_tensors='pt')
        max_length_for_generation = len(input_ids[0]) + max_response_length
        output = model.generate(
            input_ids, 
            max_length=max_length_for_generation, 
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.7,  # Adjust as needed
            top_k=50  # Adjust as needed
        )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        last_newline = response.rfind('\n')
        last_response = response[last_newline:].strip()

        # Trim the conversation to the last few lines if it becomes too long
        if len(conversation.split('\n')) > 6:
            conversation = '\n'.join(conversation.split('\n')[-3:])

        conversation_response = f"{current_character}: {last_response}"
        print(conversation_response)
        conversation += conversation_response + "\n"
        current_character = character1 if current_character == character2 else character2

    return conversation



# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", 
                                  model, tokenizer)
print(conversation)

Falstaff: What say you, merry sir, on this fine evening?


/home/galen/anaconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Bottom: 
Falstaff: The first thing I want to say to you is that I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not
Bottom: Falstaff: I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am
Falstaff: Bottom: Falstaff: I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game. I am not a fan of the game.


KeyboardInterrupt: 

In [6]:
def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, num_exchange=5, max_response_length=50):
    """
    Simulates a conversation between two Shakespearean characters with strategies to reduce repetition.

    Args:
        character1, character2, start_sentence, model, tokenizer: Same as before.
        num_exchange (int): Number of exchanges.
        max_response_length (int): Maximum length of each response.

    Returns:
        str: The simulated conversation.
    """
    stop_token = "<|endoftext|>"
    conversation = f"{character1}: {start_sentence} {stop_token}\n"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        # Use only the last part of the conversation as context
        input_context = conversation.split(stop_token)[-2].strip()
        input_ids = tokenizer.encode(input_context, return_tensors='pt')
        
        output = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + max_response_length,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.encode(stop_token)[0]
        )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        response = response.split(stop_token)[0].strip()  # Stop at the stop token

        conversation_response = f"{current_character}: {response} {stop_token}"
        print(conversation_response)
        conversation += conversation_response + "\n"
        current_character = character1 if current_character == character2 else character2

    return conversation



# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", 
                                  model, tokenizer)
print(conversation)

Falstaff: What say you, merry sir, on this fine evening? <|endoftext|>


/home/galen/anaconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Falstaff: Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Bottom: Falstaff: Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Falstaff: What say you, merry sir, on this fine evening? <|endoftext|>
Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Falstaff: Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?” <|endoftext|>
Bottom: Falstaff: Bottom: Falstaff: Bottom: Falstaff: What sa

In [7]:
def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, num_exchange=5, max_response_length=50):
    """
    Simulates a conversation between two Shakespearean characters with improved generation settings.

    Args:
        character1, character2, start_sentence, model, tokenizer: Same as before.
        num_exchange (int): Number of exchanges.
        max_response_length (int): Maximum length of each response.

    Returns:
        str: The simulated conversation.
    """
    stop_token = "<|endoftext|>"
    conversation = f"{character1}: {start_sentence} {stop_token}"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        # Use only the last part of the conversation as context
        input_context = conversation.split(stop_token)[-2].strip()
        input_ids = tokenizer.encode(input_context, return_tensors='pt')
        
        output = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + max_response_length,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.encode(stop_token)[0],
            do_sample=True  # Enable sampling
        )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        response = response.split(stop_token)[0].strip()  # Stop at the stop token

        conversation_response = f"{current_character}: {response} {stop_token}"
        print(conversation_response)
        conversation += "\n" + conversation_response
        current_character = character1 if current_character == character2 else character2

    return conversation



# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", 
                                  model, tokenizer)
print(conversation)

Falstaff: What say you, merry sir, on this fine evening? <|endoftext|>
Bottom: Falstaff: What say you, merry sir, on this fine evening? <|endoftext|>
Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?


I know that it will be nice to see the other people of my family and friends, and I am sure that I will enjoy the very best of my time.
I do not think that it will be an issue for any other <|endoftext|>
Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?


I know that it will be nice to see the other people of my family and friends, and I am sure that I will enjoy the very best of my time.
I do not think that it will be an issue for any other family or friends of my family.
I can't wait to see what the next day will be for my family.
I hope that the next day's time will be the same as my last.
Advertisements <|endoftext|>
Falstaff: Bottom: Falstaff: Bottom: Falstaff: What say you, merry sir, on this fine evening?


I know that it w

In [8]:
import random

def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, num_exchange=5):
    """
    Simulates a conversation between two Shakespearean characters with noise to prevent repetition.

    Args:
        character1 (str): Name of the first character.
        character2 (str): Name of the second character.
        start_sentence (str): The initial sentence to start the conversation.
        model: Pre-trained GPT-2 model.
        tokenizer: GPT-2 tokenizer.
        num_exchange (int): Number of exchanges between the characters.

    Returns:
        str: The simulated conversation.
    """
    conversation = f"{character1}: {start_sentence}\n"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        # Inject some randomness in temperature and top_p
        temperature = random.uniform(0.7, 1.0)
        top_p = random.uniform(0.8, 0.95)

        input_ids = tokenizer.encode(conversation, return_tensors='pt')
        output = model.generate(
            input_ids, 
            max_length=1000, 
            pad_token_id=tokenizer.eos_token_id,
            temperature=temperature,
            top_p=top_p,
            do_sample=True
        )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        last_newline = response.rfind('\n')
        last_response = response[last_newline:].strip()

        conversation_response = f"{current_character}: {last_response}"
        print(conversation_response)
        conversation += conversation_response + "\n"
        current_character = character1 if current_character == character2 else character2

    return conversation



# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", 
                                  model, tokenizer)
print(conversation)


Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: Advertisements
Bottom: My favorite part about this book is how much it's been. The story goes from the first chapter to the final chapter. There's so much to see in the story. It's so much more than you could imagine in a book. You can see in the book that all the characters are in the same person with different backgrounds, and there's so much more to see in the book that all the characters are in the same person with different backgrounds. It's so much more than you could imagine in a book. You can see in the book that all the characters are in the same person with different backgrounds, and there's so much more to see in the book that all the characters are in the same person with different backgrounds, and there's so much more to see in the book that all the characters are in the same person with different backgrounds, and there's so much more to see in the book that all the characters are in the same perso

In [9]:
import random

def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, 
                       num_exchange=5, max_response_length=50):
    """
    Simulates a conversation between two Shakespearean characters with controlled response length.

    Args:
        character1 (str): Name of the first character.
        character2 (str): Name of the second character.
        start_sentence (str): The initial sentence to start the conversation.
        model: Pre-trained GPT-2 model.
        tokenizer: GPT-2 tokenizer.
        num_exchange (int): Number of exchanges between the characters.
        max_response_length (int): Maximum length of each response.

    Returns:
        str: The simulated conversation.
    """
    conversation = f"{character1}: {start_sentence}\n"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        # Inject some randomness in temperature and top_p
        temperature = random.uniform(0.7, 1.0)
        top_p = random.uniform(0.8, 0.95)

        input_ids = tokenizer.encode(conversation, return_tensors='pt')
        # Calculate max length for the model generation
        max_length_for_generation = len(input_ids[0]) + max_response_length

        output = model.generate(
            input_ids, 
            max_length=max_length_for_generation, 
            pad_token_id=tokenizer.eos_token_id,
            temperature=temperature,
            top_p=top_p,
            do_sample=True
        )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        last_newline = response.rfind('\n')
        last_response = response[last_newline:].strip()

        conversation_response = f"{current_character}: {last_response}"
        print(conversation_response)
        conversation += conversation_response + "\n"
        current_character = character1 if current_character == character2 else character2

    return conversation

# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", model, tokenizer)
print(conversation)


Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: Battles for the
Bottom: Sell: I know you
Falstaff: Bottom
Bottom: Falstaff: I didn't get to
Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: Battles for the
Bottom: Sell: I know you
Falstaff: Bottom
Bottom: Falstaff: I didn't get to



In [10]:
import random
import re

def shakespearean_chat(character1, character2, start_sentence, model, tokenizer, num_exchange=5, max_response_length=50):
    """
    Simulates a conversation between two Shakespearean characters, ensuring complete sentences.

    Args:
        character1 (str): Name of the first character.
        character2 (str): Name of the second character.
        start_sentence (str): The initial sentence to start the conversation.
        model: Pre-trained GPT-2 model.
        tokenizer: GPT-2 tokenizer.
        num_exchange (int): Number of exchanges between the characters.
        max_response_length (int): Maximum length of each response.

    Returns:
        str: The simulated conversation.
    """
    conversation = f"{character1}: {start_sentence}\n"
    print(conversation.strip())
    current_character = character2

    for i in range(num_exchange):
        temperature = random.uniform(0.7, 1.0)
        top_p = random.uniform(0.8, 0.95)

        input_ids = tokenizer.encode(conversation, return_tensors='pt')
        max_length_for_generation = min(len(input_ids[0]) + max_response_length, 1024)

        output = model.generate(
            input_ids, 
            max_length=max_length_for_generation, 
            pad_token_id=tokenizer.eos_token_id,
            temperature=temperature,
            top_p=top_p,
            do_sample=True
        )
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        last_newline = response.rfind('\n')
        last_response = response[last_newline:].strip()

        # Trim to the last complete sentence within a reasonable length
        match = re.search(r'([.!?])[^.!?]*$', last_response)
        if match:
            end_index = match.start()
            last_response = last_response[:end_index+1]

        conversation_response = f"{current_character}: {last_response}"
        print(conversation_response)
        conversation += conversation_response + "\n"
        current_character = character1 if current_character == character2 else character2

    return conversation

# Start the conversation
conversation = shakespearean_chat("Falstaff", "Bottom", "What say you, merry sir, on this fine evening?", model, tokenizer)
print(conversation)


Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: Ludwig said, "I have no idea how many times you have received this letter. It's been a year and a half. I've heard that many times, Mr.
Bottom: Miner: But it's a long time.
Falstaff: Bottom: Mr.
Bottom: Falstaff: That
Falstaff: What say you, merry sir, on this fine evening?
Bottom: 
Falstaff: Ludwig said, "I have no idea how many times you have received this letter. It's been a year and a half. I've heard that many times, Mr.
Bottom: Miner: But it's a long time.
Falstaff: Bottom: Mr.
Bottom: Falstaff: That

